In [1]:
# Importing the necesary libraries that we are going to work with.

import pandas as pd
import yaml
from riotwatcher import LolWatcher, ApiError

In [28]:
config = yaml.load(open("riot-config.yaml"), Loader=yaml.FullLoader) # Reading in the riot-config.yaml file

lol_watcher = LolWatcher(config['API']) # Riot API Key

region = 'eun1' # Defining the region of the accounts scrape

players = [ 'xzxxsx', 'xRazvYx', 'BalonulDeAur', 'Syrebral', 'Rin Hen Tie', 'PeleBogdan', 'Damondar', 'Hit the Flash']

player_info = lol_watcher.summoner.by_name(region, 'xzxxsx') # Get player's info by their IGN. 

player_ranked_stats = lol_watcher.league.by_summoner(region, player_info['id']) # Get player's ranked stats.

player_matches = lol_watcher.match.matchlist_by_account(region, player_info['accountId']) # Get player's match history. Limited to last 100 records.

champ_list = lol_watcher.data_dragon.champions("10.15.1", False, 'en_US')['data'] # Get all available info about an in-game champion.


# For Riot's API, the 404 status code indicates that the requested data wasn't found and
# should be expected to occur in normal operation, as in the case of a an
# invalid summoner name, match ID, etc.
#
# The 429 status code indicates that the user has sent too many requests
# in a given amount of time ("rate limiting").

# Custom Error:
# try:
#     response = lol_watcher.summoner.by_name(region, 'this_is_probably_not_anyones_summoner_name')
# except ApiError as err:
#     if err.response.status_code == 429:
#         print('We should retry in {} seconds.'.format(err.response.headers['Retry-After']))
#         print('this retry-after is handled by default by the RiotWatcher library')
#         print('future requests wait until the retry-after time passes')
#     elif err.response.status_code == 404:
#         print('Summoner with that ridiculous name not found.')
#     else:
#         raise

In [6]:
# fetch match details
last_match = player_matches['matches'][0]
match_detail = lol_watcher.match.by_id(region, last_match['gameId'])
participants = []
for row in match_detail['participants']:
    participants_row = {}
    participants_row['gameID'] = last_match['gameId']
    participants_row['champion'] = row['championId']
    participants_row['spell1'] = row['spell1Id']
    participants_row['spell2'] = row['spell2Id']
    participants_row['win'] = row['stats']['win']
    participants_row['kills'] = row['stats']['kills']
    participants_row['deaths'] = row['stats']['deaths']
    participants_row['assists'] = row['stats']['assists']
    participants_row['totalDamageDealt'] = row['stats']['totalDamageDealt']
    participants_row['goldEarned'] = row['stats']['goldEarned']
    participants_row['champLevel'] = row['stats']['champLevel']
    participants_row['totalMinionsKilled'] = row['stats']['totalMinionsKilled']
    participants_row['item0'] = row['stats']['item0']
    participants_row['item1'] = row['stats']['item1']
    participants.append(participants_row)
df = pd.DataFrame(participants)
df

,gameID,champion,spell1,spell2,win,kills,deaths,assists,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1
0,2543774811,412,14,4,True,3,2,9,27909,8313,13,36,3857,3109
1,2543774811,7,4,14,True,11,5,4,95856,11275,15,145,3089,1082
2,2543774811,876,4,11,True,1,2,6,181876,9914,15,41,1402,2031
3,2543774811,875,4,14,True,4,5,7,83033,9019,14,102,1054,3078
4,2543774811,222,7,4,True,5,6,10,110655,10160,13,160,3006,3031
5,2543774811,29,4,7,False,10,5,4,79236,10736,14,124,3085,3153
6,2543774811,245,4,14,False,1,8,5,72851,7001,13,104,1056,3152
7,2543774811,104,4,11,False,3,4,7,179108,10106,14,36,1412,3006
8,2543774811,106,12,4,False,5,4,6,78024,9342,14,127,3065,3071
9,2543774811,89,4,3,False,1,3,8,29678,6597,13,52,3190,3860


In [7]:
# check league's latest version
# latest = lol_watcher.data_dragon.versions_for_region(my_region)['n']['champion']
# Lets get some champions static information
# static_champ_list = lol_watcher.data_dragon.champions("10.15.1", False, 'en_US')
# champ static list data to dict for looking up
champ_dict = {}
for key in champ_list:
    row = champ_list[key]
    champ_dict[row['key']] = row['id']
for row in participants:
    print(str(row['champion']) + ' ' + champ_dict[str(row['champion'])])
    row['championName'] = champ_dict[str(row['champion'])]
# print dataframe
df = pd.DataFrame(participants)
df

412 Thresh
7 Leblanc
876 Lillia
875 Sett
222 Jinx
29 Twitch
245 Ekko
104 Graves
106 Volibear
89 Leona


,gameID,champion,spell1,spell2,win,kills,deaths,assists,totalDamageDealt,goldEarned,champLevel,totalMinionsKilled,item0,item1,championName
0,2543774811,412,14,4,True,3,2,9,27909,8313,13,36,3857,3109,Thresh
1,2543774811,7,4,14,True,11,5,4,95856,11275,15,145,3089,1082,Leblanc
2,2543774811,876,4,11,True,1,2,6,181876,9914,15,41,1402,2031,Lillia
3,2543774811,875,4,14,True,4,5,7,83033,9019,14,102,1054,3078,Sett
4,2543774811,222,7,4,True,5,6,10,110655,10160,13,160,3006,3031,Jinx
5,2543774811,29,4,7,False,10,5,4,79236,10736,14,124,3085,3153,Twitch
6,2543774811,245,4,14,False,1,8,5,72851,7001,13,104,1056,3152,Ekko
7,2543774811,104,4,11,False,3,4,7,179108,10106,14,36,1412,3006,Graves
8,2543774811,106,12,4,False,5,4,6,78024,9342,14,127,3065,3071,Volibear
9,2543774811,89,4,3,False,1,3,8,29678,6597,13,52,3190,3860,Leona


In [111]:
users = {}

import time

for player in players:
    print(player)
    player_data = {}
    
    player_info = lol_watcher.summoner.by_name(region, player)
    
    player_data['name'] = player_info['name']
    player_data['summonerLevel'] = player_info['summonerLevel']
    
    player_matches = lol_watcher.match.matchlist_by_account(region, player_info['accountId'])['matches']
    
    player_data['games'] = []
    
    for match in player_matches:
        if match['queue'] == 440:
            game = {}
            game['gameId'] = match['gameId']
            game['champion'] = match['champion']
            game['timestamp'] = match['timestamp']
            game['role'] = match['role']
            game['lane'] = match['lane']
            game['duration'] = round(match['gameDuration'] / 60)

            player_data['games'].append(game)
#                     game['win'] = participant['stats']['win']
#                     game['K'] = participant['stats']['kills']
#                     game['D'] = participant['stats']['deaths']
#                     game['A'] = participant['stats']['assists']
#                     game['largestMultiKill'] = participant['stats']['largestMultiKill']
        
    users[player] = player_data

In [150]:
for user in users.keys():
    print(user)
    
    users[user]['stats'] = []
    
    for match in users[user]['games']:
        users[user]['matches']=[]
        game = {}
        game['gameId'] = match['gameId']
        game['champion'] = match['champion']
        game['timestamp'] = match['timestamp']
        game['role'] = match['role']
        game['lane'] = match['lane']
        game['duration'] = round(match['gameDuration'] / 60)
        
        while True:
            try:
                match_detail = lol_watcher.match.by_id(region, match['gameId'])
                print(match['gameId'])
                pass
            except:
                time.sleep(1)
                continue
            break

        for participant in match_detail['participants']:
            if match['champion'] == participant['championId']:
#                 users[user]['stats'].append(participant['stats'])
                for stat,val in participant['stats'].items():
                    game[stat]=val
            
        users[user]['stats'].append(game)

xzxxsx
2543774811
2542786996
2542119244
2542053957
2541204029
2541240350
2541175731
2541158439
2541083730
2541087611
2540238900
2540182669
2540147659
2540060174
2540006509
2539096142
2539102134
2539066262
2538478503
2538414611
2537553520
2537537218
2536738170
2536683038
2536657188
2536029378
2536014803
2536012888
2535998106
2535943059
2535196366
2534482754
2534426624
2533773192
2533810830
2533838772
2533674399
2533671285
2297372410
2297345580
2297351631
2297295073
2296467643
2295929051
2295868851
2295830631
2295754557
2295747155
2295679585
2295658765
2295461534
2295376849
2294758727
2294748133
2294164954
xRazvYx
2543774811
2543038823
2542786996
2542809961
2542119244
2542053957
2541158439
2541083730
2541087611
2540928969
2540238900
2540182669
2540147659
2540060174
2540006509
2538478503
2538414611
2537553520
2536657188
2536029378
2536014803
2536012888
2535998106
2535943059
2535196366
2534482754
2533773192
2533810830
2497467000
2497512410
2455361791
2455206303
2431147704
BalonulDeAur
2543

TypeError: list indices must be integers or slices, not str

In [148]:
pd.DataFrame(users['xRazvYx']['stats'])

,participantId,win,item0,item1,item2,item3,item4,item5,item6,kills,...,perk4Var3,perk5,perk5Var1,perk5Var2,perk5Var3,perkPrimaryStyle,perkSubStyle,statPerk0,statPerk1,statPerk2
0,7,False,1056,3152,1033,3020,1026,3108,3340,1,...,0,8106,4,0,0,8200,8100,5008,5003,5002
1,4,True,1412,3748,3142,3047,3147,2031,3340,13,...,0,8014,898,0,0,8100,8000,5005,5003,5002
2,7,False,1054,3153,3139,1011,3006,3052,3340,5,...,0,8135,1700,5,0,8000,8100,5005,5003,5002
3,7,True,1054,3748,3047,3068,1011,1031,3340,5,...,0,8136,3,6,0,8400,8100,5005,5003,5002
4,6,True,1056,3152,3020,3100,1058,0,3340,11,...,0,8106,5,0,0,8200,8100,5008,5003,5002
5,6,True,3285,3152,3157,3020,3151,0,3340,7,...,0,8106,5,0,0,8200,8100,5008,5003,5002
6,1,True,1055,3031,3006,3046,3072,3035,3340,12,...,0,8105,15,5,0,8000,8100,5008,5003,5002
7,6,True,1056,3802,3020,3157,0,1027,3340,3,...,0,8105,11,3,0,8200,8100,5008,5003,5002
8,7,True,1056,3157,3020,3285,3152,0,3340,5,...,0,8105,16,5,0,8200,8100,5008,5003,5002
9,2,False,1053,3158,3071,3133,0,0,3340,2,...,0,9104,22,50,0,8100,8000,5007,5003,5002


In [157]:
list(pd.DataFrame(users['xzxxsx']['stats']).columns)

['gameId',
 'champion',
 'timestamp',
 'role',
 'lane',
 'participantId',
 'win',
 'item0',
 'item1',
 'item2',
 'item3',
 'item4',
 'item5',
 'item6',
 'kills',
 'deaths',
 'assists',
 'largestKillingSpree',
 'largestMultiKill',
 'killingSprees',
 'longestTimeSpentLiving',
 'doubleKills',
 'tripleKills',
 'quadraKills',
 'pentaKills',
 'unrealKills',
 'totalDamageDealt',
 'magicDamageDealt',
 'physicalDamageDealt',
 'trueDamageDealt',
 'largestCriticalStrike',
 'totalDamageDealtToChampions',
 'magicDamageDealtToChampions',
 'physicalDamageDealtToChampions',
 'trueDamageDealtToChampions',
 'totalHeal',
 'totalUnitsHealed',
 'damageSelfMitigated',
 'damageDealtToObjectives',
 'damageDealtToTurrets',
 'visionScore',
 'timeCCingOthers',
 'totalDamageTaken',
 'magicalDamageTaken',
 'physicalDamageTaken',
 'trueDamageTaken',
 'goldEarned',
 'goldSpent',
 'turretKills',
 'inhibitorKills',
 'totalMinionsKilled',
 'neutralMinionsKilled',
 'neutralMinionsKilledTeamJungle',
 'neutralMinionsKille